In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import re
from collections import Counter
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool, sum_models
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle

Array = Any

In [ ]:
def load(filename):
    emdeb = np.load(filename)
    return list(emdeb)

In [ ]:
columns = ['clipTarget', 'date']
df = pd.DataFrame(columns=['clipTarget', "audio"])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    audio_filepath = f"/content/drive/MyDrive/ASR/AudioEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    temp["audio"] = load(audio_filepath)
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
sorted(set(df['date'].to_list()))

In [ ]:
df[df['date'] == '2022-07-07']

,clipTarget,audio,date
324160,SPORT,"[0.19747002, -0.09017593, -0.15468189, 0.01457...",2022-07-07
324161,TRAVEL,"[0.18064348, -0.08861546, 0.02328031, -0.07119...",2022-07-07
324162,DANCE,"[0.15282989, -0.039978504, -0.12310115, 0.0594...",2022-07-07
324163,ANIMALS,"[0.07372899, -0.08223677, -0.092304066, 0.1175...",2022-07-07
324164,FAMILY,"[0.08605196, -0.03458572, -0.12074478, -0.0027...",2022-07-07
...,...,...,...
360284,VEHICLES,"[0.15986866, 0.0019198233, -0.06104671, 0.0641...",2022-07-07
360285,NATURE,"[0.13996333, -0.13557442, -0.029511726, -0.072...",2022-07-07
360286,SOCIAL,"[0.12375446, -0.06339489, 0.02045422, -0.03073...",2022-07-07
360287,COSPLAY,"[0.19783749, -0.033241857, -0.18903485, 0.0776...",2022-07-07


In [ ]:
len(set(df.iloc[324160:, :]["clipTarget"]))

52

In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(df_train["audio"].to_list(), df_train["clipTarget"].to_list())

Learning rate set to 0.211095
0:	learn: 3.6214250	total: 283ms	remaining: 4m 42s
1:	learn: 3.4902853	total: 471ms	remaining: 3m 55s
2:	learn: 3.4042506	total: 642ms	remaining: 3m 33s
3:	learn: 3.3458026	total: 822ms	remaining: 3m 24s
4:	learn: 3.3002695	total: 996ms	remaining: 3m 18s
5:	learn: 3.2670753	total: 1.16s	remaining: 3m 12s
6:	learn: 3.2423468	total: 1.33s	remaining: 3m 8s
7:	learn: 3.2216970	total: 1.5s	remaining: 3m 5s
8:	learn: 3.2040335	total: 1.67s	remaining: 3m 3s
9:	learn: 3.1889563	total: 1.83s	remaining: 3m 1s
10:	learn: 3.1766757	total: 2s	remaining: 2m 59s
11:	learn: 3.1649278	total: 2.17s	remaining: 2m 58s
12:	learn: 3.1546520	total: 2.35s	remaining: 2m 58s
13:	learn: 3.1459746	total: 2.5s	remaining: 2m 56s
14:	learn: 3.1379371	total: 2.67s	remaining: 2m 55s
15:	learn: 3.1302395	total: 2.84s	remaining: 2m 54s
16:	learn: 3.1227224	total: 3s	remaining: 2m 53s
17:	learn: 3.1144728	total: 3.18s	remaining: 2m 53s
18:	learn: 3.1081491	total: 3.34s	remaining: 2m 52s
19:	

In [ ]:
model.save_model('/content/drive/MyDrive/ASR/Catboost/model_audio')

In [ ]:
y_preds = model.predict(df_test["audio"].to_list())

In [ ]:
f1_score(df_test["clipTarget"].to_list(), y_preds, average='macro')

0.13082230540662587

In [ ]:
y_preds_p = model.predict_proba(df_test["audio"].to_list())

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, average='macro', multi_class="ovr")

0.7705937197725021

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, average='macro', multi_class="ovo")

0.769731070640244

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, multi_class="ovr")

0.7705937197725021

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, multi_class="ovo")

0.769731070640244

In [ ]:
del df_test, df_train

In [ ]:
columns = ['clipTarget', 'date']
df = pd.DataFrame(columns=['clipTarget', "video"])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    video_filepath = f"/content/drive/MyDrive/ASR/ContentEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    temp["video"] = load(video_filepath)
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
df.sample(3)

,clipTarget,video,date
45323,FOOD,"[0.043309614, 0.0565622, -0.027773185, -0.0123...",2022-04-06
226633,CREATION,"[0.0067632035, -0.015964027, 0.07218018, 0.006...",2022-05-25
369302,HUMOR,"[-0.045005683, -0.06904212, 0.019706415, 0.010...",2022-07-15


In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(df_train["video"].to_list(), df_train["clipTarget"].to_list())
model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video')

Learning rate set to 0.211095
0:	learn: 3.3339408	total: 989ms	remaining: 16m 28s
1:	learn: 3.1029729	total: 1.77s	remaining: 14m 42s
2:	learn: 2.9674927	total: 2.57s	remaining: 14m 14s
3:	learn: 2.8746882	total: 3.46s	remaining: 14m 21s
4:	learn: 2.7947271	total: 4.36s	remaining: 14m 28s
5:	learn: 2.7479250	total: 5.15s	remaining: 14m 12s
6:	learn: 2.6993329	total: 5.94s	remaining: 14m 2s
7:	learn: 2.6529314	total: 6.84s	remaining: 14m 8s
8:	learn: 2.6043376	total: 7.7s	remaining: 14m 7s
9:	learn: 2.5674943	total: 8.56s	remaining: 14m 7s
10:	learn: 2.5369295	total: 9.39s	remaining: 14m 4s
11:	learn: 2.5057337	total: 10.3s	remaining: 14m 4s
12:	learn: 2.4850789	total: 11.1s	remaining: 14m 1s
13:	learn: 2.4554308	total: 12s	remaining: 14m 2s
14:	learn: 2.4340187	total: 12.7s	remaining: 13m 55s
15:	learn: 2.4154110	total: 13.5s	remaining: 13m 51s
16:	learn: 2.3993358	total: 14.3s	remaining: 13m 45s
17:	learn: 2.3761121	total: 15.1s	remaining: 13m 45s
18:	learn: 2.3610272	total: 15.9s	rem

In [ ]:
y_preds = model.predict(df_test["video"].to_list())

In [ ]:
f1_score(df_test["clipTarget"].to_list(), y_preds, average='macro')

0.2923139871541985

In [ ]:
print(confusion_matrix(df_test["clipTarget"].to_list(), y_preds))

[[2431    0    2 ...    7    0   21]
 [   8    2    9 ...    2    0    2]
 [   1    0   22 ...    0    0    1]
 ...
 [  11    0    0 ...  396    0   56]
 [   1    0    2 ...    0   87    1]
 [  19    0    0 ...   46    0 2198]]


In [ ]:
y_preds_p = model.predict_proba(df_test["video"].to_list())

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, average='macro', multi_class="ovr")

0.9004021954107301

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, average='macro', multi_class="ovo")

0.8861259046894993

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, multi_class="ovr")

0.9004021954107301

In [ ]:
roc_auc_score(df_test["clipTarget"].to_list(), y_preds_p, multi_class="ovo")

0.8861259046894993

In [ ]:
def load(filename):
    emdeb = np.load(filename)
    return emdeb
    
columns = ['clipTarget', 'date']
df = pd.DataFrame(columns=['clipTarget', "video_audio"])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    video_filepath = f"/content/drive/MyDrive/ASR/ContentEmbedding/0000{i}.npy"
    audio_filepath = f"/content/drive/MyDrive/ASR/AudioEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    video = load(video_filepath)
    audio = load(audio_filepath)
    temp["video_audio"] = list(np.concatenate((video, audio), axis=1))
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)
X_train = df_train["video_audio"].to_list()
y_train = df_train["clipTarget"].to_list()
del df_train

df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
X_test = df_test["video_audio"].to_list()
y_test = df_test["clipTarget"].to_list()
del df, df_test

In [ ]:
batch = Pool(X_train, label=y_train)

In [ ]:
del X_train, y_train

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(X=batch)

Learning rate set to 0.211095
0:	learn: 3.3339416	total: 1.1s	remaining: 18m 16s
1:	learn: 3.0870022	total: 2s	remaining: 16m 39s
2:	learn: 2.9536699	total: 2.92s	remaining: 16m 9s
3:	learn: 2.8588083	total: 3.84s	remaining: 15m 55s
4:	learn: 2.7756351	total: 4.86s	remaining: 16m 7s
5:	learn: 2.7050536	total: 5.82s	remaining: 16m 4s
6:	learn: 2.6483353	total: 6.76s	remaining: 15m 58s
7:	learn: 2.6065190	total: 7.56s	remaining: 15m 38s
8:	learn: 2.5719913	total: 8.41s	remaining: 15m 26s
9:	learn: 2.5282754	total: 9.32s	remaining: 15m 22s
10:	learn: 2.4897942	total: 10.3s	remaining: 15m 26s
11:	learn: 2.4684498	total: 11.2s	remaining: 15m 18s
12:	learn: 2.4401623	total: 12.1s	remaining: 15m 17s
13:	learn: 2.4125817	total: 12.9s	remaining: 15m 10s
14:	learn: 2.3823118	total: 13.8s	remaining: 15m 7s
15:	learn: 2.3536951	total: 14.8s	remaining: 15m 9s
16:	learn: 2.3292788	total: 15.8s	remaining: 15m 12s
17:	learn: 2.3087659	total: 16.7s	remaining: 15m 10s
18:	learn: 2.2880227	total: 17.5s	r

In [ ]:
model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_audio_model_sum')

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.31981002625669386

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9144301679984049

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9015344064252185

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9144301679984049

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9015344064252185